# Model Training and Evaluation

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [117]:
orig_df = pd.read_csv('../dataset/team_A_dataset.csv')

#all columns with NANs
print(orig_df.columns[orig_df.isna().any()].tolist())

#replacing NANs with zeros
df = orig_df.fillna(0)

avg_cols = ['avg_monthly_salary', 'monthly_min_wage', 'monthly_inflation_rate_wrt_last_year', 'reer', 'bilance',
            'avg_energy_price', 'avg_gasoline_price', 'avg_natural_gas_price']

avg_df = orig_df.groupby(['month', 'year'], as_index=False)[avg_cols].mean()

#sum_cols = ['general_thefts', 'break_in_thefts', 'uchazeciOZamestnaniUoZ', 'uchazeciOZamestnaniUoZZeny',
#            'noveHlaseniUchazeci', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM', 'absolventiSkolAMladistvi',
#            'z_do_65_w19', 'm_do_65', 'celkem']

#ignoring 'celkem' and 'uchazeciOZamestnaniUoZ'
sum_cols = ['general_thefts', 'break_in_thefts', 'uchazeciOZamestnaniUoZZeny',
           'noveHlaseniUchazeci', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM', 'absolventiSkolAMladistvi',
           'z_do_65_w19', 'm_do_65_w19']

sum_df = orig_df.groupby(['month', 'year'], as_index=False)[sum_cols].sum()

df = pd.merge(avg_df, sum_df, on=['month', 'year'])

df['date'] = pd.to_datetime(df['year'].astype(str) + df['month'].astype(str), format='%Y%m')

df = df.set_index('date')
df = df.sort_index()

seasonal_cols = ['avg_monthly_salary', 'general_thefts', 'break_in_thefts', 'noveHlaseniUchazeci',
                  'absolventiSkolAMladistvi', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM']

for col in seasonal_cols:
    df[col + '_prev_year'] = df[col].shift(11)

#fill previous year columns for 2009 with 2009 year values
for i in range(len(df)):
    for col in seasonal_cols:
        if np.isnan(df.loc[df.index[i], col + '_prev_year']):
            df.loc[df.index[i], col + '_prev_year'] = df.loc[df.index[i], col]

#df = df.drop(columns=['month', 'year'])
            
war_df = df.iloc[158:, :] #Since March 2022
war_df.tail(5)

['celkem', 'm_do_65', 'z_do_65', 'celkem_w2', 'celkem_w3', 'celkem_w4', 'celkem_w5', 'celkem_w6', 'celkem_w7', 'celkem_w8', 'celkem_w9', 'celkem_w10', 'celkem_w11', 'celkem_w12', 'celkem_w13', 'celkem_w14', 'celkem_w15', 'celkem_w16', 'celkem_w17', 'celkem_w18', 'celkem_w19', 'm_do_65_w2', 'm_do_65_w3', 'm_do_65_w4', 'm_do_65_w5', 'm_do_65_w6', 'm_do_65_w7', 'm_do_65_w8', 'm_do_65_w9', 'm_do_65_w10', 'm_do_65_w11', 'm_do_65_w12', 'm_do_65_w13', 'm_do_65_w14', 'm_do_65_w15', 'm_do_65_w16', 'm_do_65_w17', 'm_do_65_w18', 'm_do_65_w19', 'z_do_65_w2', 'z_do_65_w3', 'z_do_65_w4', 'z_do_65_w5', 'z_do_65_w6', 'z_do_65_w7', 'z_do_65_w8', 'z_do_65_w9', 'z_do_65_w10', 'z_do_65_w11', 'z_do_65_w12', 'z_do_65_w13', 'z_do_65_w14', 'z_do_65_w15', 'z_do_65_w16', 'z_do_65_w17', 'z_do_65_w18', 'z_do_65_w19', 'm_do_65_ratio', 'z_do_65_ratio', 'm_do_65_w2_ratio', 'm_do_65_w3_ratio', 'm_do_65_w4_ratio', 'm_do_65_w5_ratio', 'm_do_65_w6_ratio', 'm_do_65_w7_ratio', 'm_do_65_w8_ratio', 'm_do_65_w9_ratio', 'm_do

,month,year,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,bilance,avg_energy_price,avg_gasoline_price,avg_natural_gas_price,...,absolventiSkolAMladistvi,z_do_65_w19,m_do_65_w19,avg_monthly_salary_prev_year,general_thefts_prev_year,break_in_thefts_prev_year,noveHlaseniUchazeci_prev_year,absolventiSkolAMladistvi_prev_year,noveHlasenaAUvolnenaVPM_prev_year,obsazenaAZrusenaVPM_prev_year
date,,,,,,,,,,,,,,,,,,,,,
2023-05-01,5,2023,40930.142857,17300.0,11.1,117.36,9.370,150.59,1.6911,2.3395,...,11499,149692.0,82353.0,37914.357143,2927.0,2690.0,30851.0,8999.0,23787.0,41710.0
2023-06-01,6,2023,40930.142857,17300.0,9.7,117.36,19.236,150.19,1.6982,2.4959,...,9512,153834.0,84692.0,37790.571429,3201.0,2894.0,34846.0,9446.0,17232.0,23390.0
2023-07-01,7,2023,40473.285714,17300.0,8.8,115.41,-6.421,156.26,1.7325,2.6531,...,9928,158260.0,87720.0,37790.571429,3325.0,2912.0,42324.0,10420.0,22812.0,23735.0
2023-08-01,8,2023,40473.285714,17300.0,8.5,115.41,-5.628,152.26,1.7791,2.6613,...,10795,161187.0,89998.0,37790.571429,2969.0,2763.0,49340.0,15490.0,20650.0,26879.0
2023-09-01,9,2023,40473.285714,17300.0,6.9,115.41,11.874,138.98,1.7623,2.7372,...,16894,164537.0,92435.0,41179.142857,3075.0,3114.0,38161.0,15117.0,19413.0,28151.0


## Since start of the war refugees predictor present

In [114]:
n_splits = len(war_df) - 1
X = war_df.loc[:, war_df.columns != 'uchazeciOZamestnaniUoZZeny'].to_numpy()
y = war_df.loc[:, war_df.columns == 'uchazeciOZamestnaniUoZZeny'].to_numpy()

q = 0.95
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))

best_err = [[10e9]]
best_alpha = -1
for alpha in np.logspace(-3, 3, num=7, base=10):
    weighted_error = 0
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)
    for train_index, test_index in tscv.split(X):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_index, :])
        X_test = scaler.transform(X[test_index, :])

        model = Lasso(alpha=alpha, max_iter=10000000)
        model.fit(X_train, y[train_index])

        y_pred = model.predict(X_test)
        y_target = y[test_index]
        weighted_error += abs(y_target - y_pred) * weights[test_index-1]

    if weighted_error < best_err:
        best_err = weighted_error
        best_alpha = alpha

print("Best absolute weighted_error " + str(best_err[0][0]) + " for alpha: " + str(alpha))
    

/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+

Best absolute weighted_error 38187.93032146147 for alpha: 1000.0


/home/david/.local/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(


## Entire period without refugees

In [116]:
n_splits = len(war_df) - 1
X = df.loc[:, np.invert(df.columns.isin(['uchazeciOZamestnaniUoZZeny', 'z_do_65_w19', 'm_do_65_w19']))].to_numpy()
y = df.loc[:, df.columns == 'uchazeciOZamestnaniUoZZeny'].to_numpy()

q = 0.95
exps = np.linspace(0, n_splits-1, num=n_splits)
weights = np.flip(np.power(q, exps))

best_err = [[10e9]]
best_alpha = -1
for alpha in np.logspace(-3, 3, num=7, base=10):
    weighted_error = 0
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=1)
    for train_index, test_index in tscv.split(X):
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X[train_index, :])
        X_test = scaler.transform(X[test_index, :])

        model = Lasso(alpha=alpha, max_iter=100000000)
        model.fit(X_train, y[train_index])

        y_pred = model.predict(X_test)
        y_target = y[test_index]
        weighted_error += abs(y_target - y_pred) * weights[test_index-1-len(df)+len(war_df)]

    if weighted_error < best_err:
        best_err = weighted_error
        best_alpha = alpha

print("Best absolute weighted_error " + str(best_err[0][0]) + " for alpha: " + str(alpha))

Best absolute weighted_error 60078.304464707944 for alpha: 1000.0


# Mlynatom experiments

## Prepare Data

In [6]:
orig_df = pd.read_csv('../dataset/team_A_dataset.csv')
orig_df.head

<bound method NDFrame.head of       month  year kraj  general_thefts  break_in_thefts  avg_monthly_salary  \
0         1  2009  HKK           271.0            174.0             19132.0   
1         1  2009  JHC           275.0            181.0             19576.0   
2         1  2009  JHM           909.0            286.0             21065.0   
3         1  2009  KVK           198.0             96.0             18652.0   
4         1  2009  LBK           343.0            130.0             19653.0   
...     ...   ...  ...             ...              ...                 ...   
2473      9  2023  PLK           145.0            130.0             41220.0   
2474      9  2023  STC           262.0            254.0             42990.0   
2475      9  2023  ULK           243.0            200.0             39664.0   
2476      9  2023  VYS            62.0             39.0             39315.0   
2477      9  2023  ZLK            67.0             57.0             38596.0   

      celkem  m_do_65

In [10]:
display(orig_df.isna().sum())

month                               0
year                                0
kraj                                0
general_thefts                      0
break_in_thefts                     0
                                   ..
noveHlasenaAUvolnenaVPM             0
obsazenaAZrusenaVPM                 0
absolventiSkolAMladistvi            0
uchazeciOZamestnaniUoZMuzi_ratio    0
uchazeciOZamestnaniUoZZeny_ratio    0
Length: 117, dtype: int64

In [11]:
orig_df.describe()

,month,year,general_thefts,break_in_thefts,avg_monthly_salary,celkem,m_do_65,z_do_65,celkem_w2,celkem_w3,...,avg_natural_gas_price,uchazeciOZamestnaniUoZ,uchazeciOZamestnaniUoZZeny,uchazeciOZamestnaniUoZMuzi,noveHlaseniUchazeci,noveHlasenaAUvolnenaVPM,obsazenaAZrusenaVPM,absolventiSkolAMladistvi,uchazeciOZamestnaniUoZMuzi_ratio,uchazeciOZamestnaniUoZZeny_ratio
count,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,266.000000,266.000000,266.000000,266.000000,266.000000,...,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000,2478.000000
mean,6.423729,2015.881356,489.349475,233.924939,28453.470944,1394.827068,347.500000,618.560150,2762.872180,4105.748120,...,3.499783,27967.878531,14174.182809,13793.695722,3371.029459,1975.188458,1899.223164,1480.798628,0.486333,0.513667
std,3.429731,4.259787,681.754931,214.294818,6730.770886,5885.779007,785.677495,2600.293961,8979.240558,11466.193698,...,1.236720,17887.574100,8725.674030,9234.474479,1770.417610,1716.588043,1706.252205,1062.353914,0.030833,0.030833
min,1.000000,2009.000000,19.000000,19.000000,17704.000000,-26447.000000,-3008.000000,-12127.000000,-24614.000000,-22643.000000,...,1.714700,5329.000000,2808.000000,2413.000000,817.000000,0.000000,0.000000,165.000000,0.408138,0.431466
25%,3.000000,2012.000000,139.000000,80.000000,22815.000000,276.250000,104.250000,125.250000,543.250000,801.250000,...,2.664000,13350.250000,6925.750000,6407.500000,1965.000000,883.250000,893.250000,633.000000,0.464473,0.491328
50%,6.000000,2016.000000,252.000000,162.000000,26431.500000,509.500000,177.500000,247.500000,1072.500000,1629.000000,...,3.214300,23996.000000,12384.500000,11521.000000,2892.000000,1529.000000,1450.000000,1244.000000,0.486783,0.513217
75%,9.000000,2020.000000,479.000000,282.000000,33427.000000,1317.000000,446.500000,660.000000,3221.250000,5672.750000,...,4.080000,34709.000000,17910.500000,17183.500000,4402.500000,2536.250000,2360.000000,1895.000000,0.508672,0.535527
max,12.000000,2023.000000,4599.000000,1107.000000,53070.000000,60636.000000,6854.000000,27163.000000,74113.000000,83545.000000,...,8.770500,93714.000000,43201.000000,50572.000000,12418.000000,12114.000000,17285.000000,6058.000000,0.568534,0.591862


In [12]:
orig_df.columns

Index(['month', 'year', 'kraj', 'general_thefts', 'break_in_thefts',
       'avg_monthly_salary', 'celkem', 'm_do_65', 'z_do_65', 'celkem_w2',
       ...
       'avg_natural_gas_price', 'uchazeciOZamestnaniUoZ',
       'uchazeciOZamestnaniUoZZeny', 'uchazeciOZamestnaniUoZMuzi',
       'noveHlaseniUchazeci', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM',
       'absolventiSkolAMladistvi', 'uchazeciOZamestnaniUoZMuzi_ratio',
       'uchazeciOZamestnaniUoZZeny_ratio'],
      dtype='object', length=117)

### Dataset version 1 - predicting uchazeciOZamestnaniUoZZeny_ratio, all rows, without refugee info -> big (economic) model

In [31]:
drop_cols = [f"celkem_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}" for w in range(2,20)]
drop_cols += [f"z_do_65_w{w}" for w in range(2,20)]
drop_cols += [f"m_do_65_w{w}_ratio" for w in range(2,20)]
drop_cols += [f"z_do_65_w{w}_ratio" for w in range(2,20)]
drop_cols += ["celkem", "m_do_65", "z_do_65","m_do_65_ratio", "z_do_65_ratio", "uchazeciOZamestnaniUoZ",	"uchazeciOZamestnaniUoZZeny", "uchazeciOZamestnaniUoZMuzi", "uchazeciOZamestnaniUoZMuzi_ratio"]


In [32]:
df1 = orig_df.drop(columns=drop_cols)
display(df1.columns)
display(df1.info())

Index(['month', 'year', 'kraj', 'general_thefts', 'break_in_thefts',
       'avg_monthly_salary', 'monthly_min_wage',
       'monthly_inflation_rate_wrt_last_year', 'reer', 'bilance',
       'avg_energy_price', 'avg_gasoline_price', 'avg_natural_gas_price',
       'noveHlaseniUchazeci', 'noveHlasenaAUvolnenaVPM', 'obsazenaAZrusenaVPM',
       'absolventiSkolAMladistvi', 'uchazeciOZamestnaniUoZZeny_ratio'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   month                                 2478 non-null   int64  
 1   year                                  2478 non-null   int64  
 2   kraj                                  2478 non-null   object 
 3   general_thefts                        2478 non-null   float64
 4   break_in_thefts                       2478 non-null   float64
 5   avg_monthly_salary                    2478 non-null   float64
 6   monthly_min_wage                      2478 non-null   int64  
 7   monthly_inflation_rate_wrt_last_year  2478 non-null   float64
 8   reer                                  2478 non-null   float64
 9   bilance                               2478 non-null   float64
 10  avg_energy_price                      2478 non-null   float64
 11  avg_gasoline_pric

None

categorical column is kraj (month and year are probably not - in terms of time series)

In [33]:
from sklearn.preprocessing import OneHotEncoder


encoder = OneHotEncoder(handle_unknown="ignore", sparse_output=False, drop="first")

obj_cols = df1.select_dtypes('object')
encoder.fit(obj_cols)

def transform_df(df: pd.DataFrame, encoder: OneHotEncoder) -> pd.DataFrame:
    # select all object columns
    obj_cols = df.select_dtypes('object')
    # apply transformation by encoder fitted on the training split
    transformed_cols = encoder.transform(obj_cols)
    # get new feature names
    feature_names = encoder.get_feature_names_out()
    # create new dataframe
    transformed_df = pd.DataFrame(
        transformed_cols, index=df.index, columns=feature_names).astype(bool)
    new_df = pd.concat(
        [df.select_dtypes(exclude='object'), transformed_df], axis=1)
    return new_df


df1 = transform_df(df1, encoder=encoder)

In [34]:
df1.head()

,month,year,general_thefts,break_in_thefts,avg_monthly_salary,monthly_min_wage,monthly_inflation_rate_wrt_last_year,reer,bilance,avg_energy_price,...,kraj_LBK,kraj_MSK,kraj_OLK,kraj_PAK,kraj_PHA,kraj_PLK,kraj_STC,kraj_ULK,kraj_VYS,kraj_ZLK
0,1,2009,271.0,174.0,19132.0,8000,2.2,89.45,-2.73,70.209,...,False,False,False,False,False,False,False,False,False,False
1,1,2009,275.0,181.0,19576.0,8000,2.2,89.45,-2.73,70.209,...,False,False,False,False,False,False,False,False,False,False
2,1,2009,909.0,286.0,21065.0,8000,2.2,89.45,-2.73,70.209,...,False,False,False,False,False,False,False,False,False,False
3,1,2009,198.0,96.0,18652.0,8000,2.2,89.45,-2.73,70.209,...,False,False,False,False,False,False,False,False,False,False
4,1,2009,343.0,130.0,19653.0,8000,2.2,89.45,-2.73,70.209,...,True,False,False,False,False,False,False,False,False,False


In [35]:
df1.isna().sum()

month                                   0
year                                    0
general_thefts                          0
break_in_thefts                         0
avg_monthly_salary                      0
monthly_min_wage                        0
monthly_inflation_rate_wrt_last_year    0
reer                                    0
bilance                                 0
avg_energy_price                        0
avg_gasoline_price                      0
avg_natural_gas_price                   0
noveHlaseniUchazeci                     0
noveHlasenaAUvolnenaVPM                 0
obsazenaAZrusenaVPM                     0
absolventiSkolAMladistvi                0
uchazeciOZamestnaniUoZZeny_ratio        0
kraj_JHC                                0
kraj_JHM                                0
kraj_KVK                                0
kraj_LBK                                0
kraj_MSK                                0
kraj_OLK                                0
kraj_PAK                          

### Dataset version 2 - predicting uchazeciOZamestnaniUoZZeny_ratio, refugees rows -> refugee model

### Dataset version 3 - predicting uchazeciOZamestnaniUoZZeny_ratio, all rows, imputation on refugee rows in years before -> refugee model

### Other versions can be for ratio of men

### Ridge Regression